<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - List all comments and reactions from Open Issues on a given repository
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Add+new+issues+as+page+in+Notion+database:+Error+short+description">Bug report</a>

**Tags:** #github #openIssues #reaction #comment #creator #assignees #automation

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-08 (Created: 2024-06-04)

**Description:** This notebook allows users to retrieve all comments and reactions from open issues along with creator and assignee data on a given repository.

## Input

In [68]:
import requests
import re
import pandas as pd
import naas_python as naas
from urllib.parse import urlparse
import time

### Setup variables
- `github_token`: personal token creates
- `github_url`: link to the chosen github repo
- `state`: type of issue

In [69]:
github_token = naas.secret.get("GITHUB_TOKEN").value
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
state = 'open'

## Model

### Filter on open issues without pull requests

In [70]:
def get_filtered_open_issues(github_url, personal_access_token):
    # Parse the GitHub URL to get the owner and repo name
    parsed_url = urlparse(github_url)
    path_parts = parsed_url.path.strip('/').split('/')
    
    if len(path_parts) != 2:
        raise ValueError("Invalid GitHub URL. It should be in the format: https://github.com/owner/repo")
    
    owner, repo = path_parts
    
    # GitHub API URL for issues
    api_url = f"https://api.github.com/repos/{owner}/{repo}/issues"
    
    # Set up headers with personal access token for authentication
    headers = {
        'Authorization': f'token {personal_access_token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    # Set up parameters to get only open issues
    params = {
        'state': state,
        'filter': 'all',
        'per_page': 100  # Set the number of issues per page to 100 (maximum allowed by GitHub API)
    }
    
    # Initialize an empty list to store all open issues excluding pull requests
    filtered_issues = []
    
    # Pagination to fetch all issues
    page = 1
    while True:
        params['page'] = page
        response = requests.get(api_url, headers=headers, params=params)
        
        if response.status_code != 200:
            raise Exception(f"Error fetching issues from GitHub: {response.status_code} {response.reason}")
        
        issues = response.json()
        if not issues:
            break
        
        # Filter out pull requests from the issues
        for issue in issues:
            if 'pull_request' not in issue:
                filtered_issues.append(issue)
        
        page += 1
    
    return filtered_issues


filtered_issues = get_filtered_open_issues(repo_url, github_token)

### Get comments and reactions from open issues

In [71]:
def get_all_comments(api_url, personal_access_token):
    # Set up headers with personal access token for authentication
    headers = {
        'Authorization': f'token {personal_access_token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    # Get comments for the issue
    response = requests.get(api_url + '/comments', headers=headers)
    if response.status_code != 200:
        raise Exception(f"Error fetching comments from GitHub: {response.status_code} {response.reason}")
    comments = response.json()

    return comments

def get_comment_reactions(api_url, personal_access_token):
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.squirrel-girl-preview+json'  # Required for reactions API
    }

    response = requests.get(api_url, headers=headers)
    response = requests.get(api_url + '/reactions', headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch reactions for comment {comment_id}: {response.status_code}")
        return []

### Get creator and assignees from open issues

In [75]:
creator_assignee_data = []

for issue in filtered_issues:
    # Creator
    issue_number = issue['number']
    issue_id = issue['id']
    issue_type = "CREATOR"
    issue_content = issue['title']
    issue_time = issue ['created_at']
    issue_user_name = issue['user']['name'] if 'name' in issue['user'] else 'N/A'
    issue_user_login = issue['user']['login']
    issue_user_id = issue['user']['id']
    
    creator_assignee_data.append({
        "ISSUE_NUMBER": issue_number,
        "ID": issue_id,
        "TYPE": issue_type,
        "CONTENT": issue_content,
        "DATE_TIME": issue_time,
        "USER_NAME": issue_user_name,
        "USER_LOGIN": issue_user_login,
        "USER_ID": issue_user_id,
    })

   
    # Assignees
    if (issue['assignees'] != []):
        for assignee in issue['assignees']:
            issue_number = issue['number']
            issue_id = issue['id']
            issue_type = "ASSIGNEE"
            issue_content = issue['title']
            issue_time = issue ['updated_at']
            issue_user_name = assignee['name'] if 'name' in assignee else 'N/A'
            issue_user_login = assignee['login']
            issue_user_id = assignee['id']

            creator_assignee_data.append({
                "ISSUE_NUMBER": issue_number,
                "ID": issue_id,
                "TYPE": issue_type,
                "CONTENT": issue_content,
                "DATE_TIME": issue_time,
                "USER_NAME": issue_user_name,
                "USER_LOGIN": issue_user_login,
                "USER_ID": issue_user_id,
            })

## Output
### Display result

In [73]:
issue_data = []

for issue in filtered_issues:
    api_url = issue['url']
    comments = get_all_comments(api_url, github_token)
    
    if (comments == []):
        issue_data.append({
            "ISSUE_NUMBER": issue['number'],
            "ID": "N/A",
            "TYPE": "COMMENT",
            "CONTENT": "N/A",
            "DATE_TIME": "N/A",
            "USER_NAME": "N/A",
            "USER_LOGIN": "N/A",
            "USER_ID": "N/A",
        })

    else:
        for comment in comments:
            issue_number = issue['number']
            comment_id = comment['id']
            comment_body = comment['body']
            comment_created_at = comment['created_at']
            comment_user_name = comment['user']['name'] if 'name' in comment['user'] else 'N/A'
            comment_user_login = comment['user']['login']
            comment_user_id = comment['user']['id']

            issue_data.append({
                "ISSUE_NUMBER": issue_number,
                "ID": comment_id,
                "TYPE": "COMMENT",
                "CONTENT": comment_body,
                "DATE_TIME": comment_created_at,
                "USER_NAME": comment_user_name,
                "USER_LOGIN": comment_user_login,
                "USER_ID": comment_user_id,
            })
            
            reactions = get_comment_reactions(api_url, github_token)
            if (reactions == []):
                issue_data.append({
                    "ISSUE_NUMBER": issue['number'],
                    "ID": "N/A",
                    "TYPE": "COMMENT",
                    "CONTENT": "N/A",
                    "DATE_TIME": "N/A",
                    "USER_NAME": "N/A",
                    "USER_LOGIN": "N/A",
                    "USER_ID": "N/A",
                })
            else:
                for reaction in reactions:
                    issue_number = issue['number']
                    reaction_id = reaction['id']
                    reaction_body = reaction['content']
                    reaction_created_at = reaction['created_at']
                    reaction_user_name = reaction['user']['name'] if 'name' in comment['user'] else 'N/A'
                    reaction_user_login = reaction['user']['login']
                    reaction_user_id = reaction['user']['id']

                    issue_data.append({
                        "ISSUE_NUMBER": issue['number'],
                        "ID": comment_id,
                        "TYPE": "REACTION",
                        "CONTENT": reaction_body,
                        "DATE_TIME": reaction_created_at,
                        "USER_NAME": reaction_user_name,
                        "USER_LOGIN": reaction_user_login,
                        "USER_ID": reaction_user_id,
                    })
    
df = pd.DataFrame(issue_data)
df

,ISSUE_NUMBER,ID,TYPE,CONTENT,DATE_TIME,USER_NAME,USER_LOGIN,USER_ID
0,2535,N/A,COMMENT,N/A,N/A,N/A,N/A,N/A
1,2530,N/A,COMMENT,N/A,N/A,N/A,N/A,N/A
2,2529,N/A,COMMENT,N/A,N/A,N/A,N/A,N/A
3,2528,2144513169,COMMENT,"@varshakumarr , you can create multiples templ...",2024-06-03T07:53:08Z,N/A,FlorentLvr,48032461
4,2528,N/A,COMMENT,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...
517,98,1046289981,REACTION,heart,2021-12-14T17:22:13Z,N/A,iair,5262663
518,98,1074033046,COMMENT,@iair anything we can do? We think we need to ...,2022-03-21T15:20:59Z,N/A,jravenel,21052349
519,98,1074033046,REACTION,+1,2021-12-14T17:22:06Z,N/A,iair,5262663
520,98,1074033046,REACTION,rocket,2021-12-14T17:22:08Z,N/A,iair,5262663


In [74]:
df2 = pd.DataFrame(creator_assignee_data)
df2

,ISSUE_NUMBER,ID,TYPE,CONTENT,DATE_TIME,USER_NAME,USER_LOGIN,USER_ID
0,2535,2337718785,CREATOR,GitHub - List all comments and reactions from ...,2024-06-06T08:53:47Z,N/A,FlorentLvr,48032461
1,2535,2337718785,ASSIGNEE,GitHub - List all comments and reactions from ...,2024-06-06T09:00:15Z,N/A,varshakumarr,168480119
2,2530,2330432897,CREATOR,GitHub - Get comments and reactions from issue,2024-06-03T07:54:37Z,N/A,FlorentLvr,48032461
3,2530,2330432897,ASSIGNEE,GitHub - Get comments and reactions from issue,2024-06-06T09:16:08Z,N/A,varshakumarr,168480119
4,2529,2330430872,CREATOR,GitHub - Get reactions from comment,2024-06-03T07:53:32Z,N/A,FlorentLvr,48032461
...,...,...,...,...,...,...,...,...
227,183,972474501,ASSIGNEE,Naas Manager - Add Notebook controls in CI,2022-02-22T14:31:57Z,N/A,Dr0p42,22365519
228,102,909945531,CREATOR,Google Fit - Get steps from API,2021-06-02T22:53:17Z,N/A,jravenel,21052349
229,102,909945531,ASSIGNEE,Google Fit - Get steps from API,2023-05-16T06:59:32Z,N/A,himanshu007-creator,65963997
230,98,909939228,CREATOR,Fitbit - Connect to API,2021-06-02T22:39:14Z,N/A,jravenel,21052349
